<a href="https://colab.research.google.com/github/Sanjivkumar100/ZeoTap/blob/main/Sanjiv_Kumar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
customers = pd.read_csv('Customers.csv')
transactions= pd.read_csv('Transactions.csv')


In [14]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB


In [3]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    1000 non-null   object 
 1   CustomerID       1000 non-null   object 
 2   ProductID        1000 non-null   object 
 3   TransactionDate  1000 non-null   object 
 4   Quantity         1000 non-null   int64  
 5   TotalValue       1000 non-null   float64
 6   Price            1000 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 54.8+ KB


In [48]:
transactionssummary=transactions.groupby('CustomerID').agg({'ProductID':'nunique','TotalValue':'sum','TransactionID':'count'}).reset_index()
transactionssummary.rename(columns={'ProductID':'UniqueProducts','TotalValue':'TotalSpend','TransactionID':'TransactionCounts'},inplace=True)
customers=pd.merge(customers,transactionssummary,on='CustomerID',how='left')

In [51]:
customers.fillna({
    "TotalSpend": 0,
    "TransactionCount": 0,
    "UniqueProducts": 0
}, inplace=True)



In [52]:
customers.isna().sum()

,0
CustomerID,0
CustomerName,0
Region,0
SignupDate,0
UniqueProducts,0
TotalSpend,0
TransactionCounts,1


In [55]:
mean_value = customers["TransactionCounts"].mean()
customers["TransactionCounts"].fillna(mean_value, inplace=True)


<ipython-input-55-9d4979c1087b>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customers["TransactionCounts"].fillna(mean_value, inplace=True)


In [56]:
customers.isna().sum()

,0
CustomerID,0
CustomerName,0
Region,0
SignupDate,0
UniqueProducts,0
TotalSpend,0
TransactionCounts,0


In [57]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()

customerregioncoded=ohe.fit_transform(customers[['Region']])
customerregioncoded_df=pd.DataFrame(customerregioncoded.toarray(),columns=ohe.get_feature_names_out(['Region']))
customers=pd.concat([customers.drop(['Region'],axis=1),customerregioncoded_df],axis=1)

In [44]:
customers.isna().

,0
CustomerID,0
UniqueProducts,0
TotalSpend,0
TransactionCounts,1
Region_Asia,0
Region_Europe,0
Region_North America,0
Region_South America,0


In [58]:
customer=customers.drop(columns=['SignupDate','CustomerName'], inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
numerical_columns=['TotalSpend','TransactionCounts','UniqueProducts']
customers['']=scaler.fit_transform(customers[['Age']])

In [59]:
from sklearn.metrics.pairwise import cosine_similarity

def getlookalike(input_id, data, top=3):
  customer_id = data['CustomerID']
  customer_profiles = data.drop(columns=['CustomerID'])

  input_idx = customer_id[customer_id == input_id].index[0]
  input_profile = customer_profiles.iloc[[input_idx]].values.reshape(1, -1)
  similarityscore = cosine_similarity(input_profile, customer_profiles).flatten()
  similarityscore_df = sorted([(customer_id[i], score) for i, score in enumerate(similarityscore) if i != input_idx], key=lambda x: x[1], reverse=True)
  return similarityscore_df[:top]

lookalike_map = {}
for customer_id in customers['CustomerID']:
  lookalike = getlookalike(customer_id, customers)
  lookalike_map[customer_id] = lookalike

lookalike_data = []

for cust_id, lookalike in lookalike_map.items():
  row = [cust_id]
  for lookalike_id, score in lookalike:
    row.extend([lookalike_id, score])
  lookalike_data.append(row)

columns = ['custid', 'lookalike1', 'score1', 'lookalike2', 'score2', 'lookalike3', 'score3']
lookalike_df = pd.DataFrame(lookalike_data, columns=columns)
lookalike_df.to_csv('lookalike.csv', index=False)